# Søk om utsatt frist for et set av klienter

In [ ]:
from docs.test.testinnsending.skatteetaten_api import get_access_token
from requests import post
from faker import Faker
import jwt
import uuid

token = get_access_token()
jwt_token = token["Authorization"].replace("Bearer ", "")

In [ ]:
def lag_indentifikator(identifikator: str) -> dict():
    fake = Faker()

    if len(identifikator) > 9:
        identifikator_dict = {"foedselsnummer" : identifikator}
    else:
        identifikator_dict = {"organisasjonsnummer" : identifikator}

    kandidat = {
        "identifikator": identifikator_dict,
        "naering":  True,
        "navn": fake.name(),
        "epost": fake.email(),
    }
    return kandidat

with open("utsatt_frist_kandidater.csv") as f:
    parter = []

    for line in f:
        fnr, partn = line.split(";")
        parter.append(fnr)

klienter = [lag_indentifikator(kandidat) for kandidat in parter]
payload = {"referanse": str(uuid.uuid4()),
           "klienter": klienter}
inntektsår = "2024"
#henter innlogget bruker fra tokenet
identifikator = jwt.decode(jwt_token, options={"verify_signature": False})["sub"]
url_path = f"/api/skattemelding/v2/utsattfristsoeknad/{identifikator}"
#url = "http://ekstern-token-validering.fastsetting.formueinntekt.skattemelding.sit.apps.utv10.paas.skead.no"
base_url = "https://idporten-api-sbstest.sits.no"

In [ ]:
utsatt_frist_request = post(base_url+url_path, headers=token, json=payload)
print(f"http: {utsatt_frist_request.status_code}")
print(utsatt_frist_request.text)